In [1]:
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient
login(token=UserSecretsClient().get_secret('HF_TOKEN'))

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [2]:
!pip install qwen-vl-utils
!pip install -q git+https://github.com/huggingface/transformers

In [1]:
!nvidia-smi

Sat Sep 14 14:46:05 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.90.07              Driver Version: 550.90.07      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   39C    P8              9W /   70W |       1MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info
import torch

model = Qwen2VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2-VL-7B-Instruct", torch_dtype=torch.float16, device_map="auto"
)

config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

Unrecognized keys in `rope_scaling` for 'rope_type'='default': {'mrope_section'}


model.safetensors.index.json:   0%|          | 0.00/56.5k [00:00<?, ?B/s]

model-00001-of-00005.safetensors:   0%|          | 0.00/3.90G [00:00<?, ?B/s]

model-00002-of-00005.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

...

model-00004-of-00005.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00005-of-00005.safetensors:   0%|          | 0.00/1.09G [00:00<?, ?B/s]

`Qwen2VLRotaryEmbedding` can now be fully parameterized by passing the model config through the `config` argument. All other arguments will be removed in v4.46


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/244 [00:00<?, ?B/s]

In [4]:
min_pixels = 256*28*28
max_pixels = 1280*28*28
processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-7B-Instruct", min_pixels=256*28*28, max_pixels=512*28*28)

preprocessor_config.json:   0%|          | 0.00/347 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

In [5]:
import pandas as pd
import pandas as pd
import requests
from PIL import Image
from io import BytesIO
import re

In [6]:
test_df = pd.read_csv('/kaggle/input/test-main/test.csv')

In [116]:
test_df.head()

,index,image_link,group_id,entity_name
0,0,https://m.media-amazon.com/images/I/110EibNycl...,156839,height
1,1,https://m.media-amazon.com/images/I/11TU2clswz...,792578,width
2,2,https://m.media-amazon.com/images/I/11TU2clswz...,792578,height
3,3,https://m.media-amazon.com/images/I/11TU2clswz...,792578,depth
4,4,https://m.media-amazon.com/images/I/11gHj8dhhr...,792578,depth


In [7]:
entity_unit_map = {
                        'width': 
                        {
                            'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'
                        },
                        'depth': {
                            'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'
                        },
                        'height': {
                            'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'
                        },
                        'item_weight': {
                            'gram', 'kilogram', 'microgram', 'milligram', 'ounce', 'pound', 'ton'
                        },
                        'maximum_weight_recommendation': {
                            'gram', 'kilogram', 'microgram', 'milligram', 'ounce', 'pound', 'ton'
                        },
                        'voltage': {
                            'kilovolt', 'millivolt', 'volt'
                        },
                        'wattage': {
                            'kilowatt', 'watt'
                        },
                        'item_volume': {
                            'centilitre', 'cubic foot', 'cubic inch', 'cup', 'decilitre',
                            'fluid ounce', 'gallon', 'imperial gallon', 'litre', 'microlitre',
                            'millilitre', 'pint', 'quart'
                        }
                    }

In [8]:
def map_unit(unit, entity_name, entity_unit_map):
    # Define a detailed unit mapping for common units and their conversions
    unit_conversion = {
        # Length units
        'millimetre': 'millimetre',
        'mm': 'millimetre',
        'cm': 'centimetre',
        'centimetre': 'centimetre',
        'meter': 'metre',
        'm': 'metre',
        'metre':'metre',
        'yard': 'yard',
        'foot': 'foot',
        'inch': 'inch',
        'inches':'inch',
        
        # Weight units
        'gram': 'gram',
        'g': 'gram',
        'grams':'gram',
        'serving': 'gram',
        'kilogram': 'kilogram',
        'kg': 'kilogram',
        'kgs': 'kilogram',
        'microgram': 'microgram',
        'mg': 'milligram',
        'milligrams': 'milligram',
        'ounce': 'ounce',
        'oz':'ounce',
        'pound': 'pound',
        'pounds': 'pound',
        'lbs': 'pound',
        'ton': 'ton',
        
        # Volume units
        'fluid ounce': 'fluid ounce',
        'fl oz': 'fluid ounce',
        'cup': 'cup',
        'gallon': 'gallon',
        'quart': 'quart',
        'pint': 'pint',
        'liter': 'litre',
        'litre': 'litre',
        'millilitre': 'millilitre',
        'ml': 'millilitre',
        
        # Voltage units
        'volt': 'volt',
        'v': 'volt',
        'kilovolt': 'kilovolt',
        'kv': 'kilovolt',
        'millivolt': 'millivolt',
        'mv': 'millivolt',
        
        # Wattage units
        'watt': 'watt',
        'w': 'watt',
        'kilowatt': 'kilowatt',
        'kw': 'kilowatt',
    }

    # Standardize the unit from the image
    standardized_unit = unit_conversion.get(unit.lower(), None)
    
    # Check if the standardized unit is allowed for the given entity
    if standardized_unit and entity_name in entity_unit_map:
        allowed_units = entity_unit_map[entity_name]
        if standardized_unit in allowed_units:
            return standardized_unit

    return None

def parse_output(output, entity_name, entity_unit_map):
    # Regular expression to extract number and unit
    match = re.match(r'(\d+\.?\d*)\s*(\w+)', output.strip())
    if match:
        number_str, unit = match.groups()
        try:            
            # Map the unit using the entity_unit_map
            mapped_unit = map_unit(unit, entity_name, entity_unit_map)
            
             # If the unit is 'kg', keep the number as an integer, else convert to float
            if unit.lower() == 'inch' or unit.lower() == 'volt':
                number = float(number_str)  # Convert to integer representation if needed
            else:
                number = int(number_str)  # Convert to float for other units
            
            if mapped_unit:
                return f"{number} {mapped_unit}"
            else:
                return ""
        except ValueError:
            return ""
    return ""


In [ ]:
# Iterate through the DataFrame and process the images
from tqdm.notebook import tqdm
import requests
from PIL import Image
from io import BytesIO

i = 40000
j = 50000
for index, row in tqdm(test_df[i:j].iterrows(), total=len(test_df[i:j])):
    img_url = row['image_link']
    entity_name = row['entity_name']
    
    try:
        # Try to download the image from the link
        response = requests.get(img_url)
        response.raise_for_status()  # Check if the image link is valid
        img = Image.open(BytesIO(response.content))

        # Prepare the image and text inputs for the QWEN2 model
        messages = [
            {
                "role": "system",
                "content": """
                    You are tasked with extracting entity values from images and formatting them strictly as “x unit”, 
                    where "x" is a float number in standard formatting, and "unit" is one of the allowed units in the 
                    provided entity_unit_map. follow this strictly.
                """
            },
            {
    "role": "user",
    "content": [
        {
            "type": "image",
            "image": f"{img_url}",  # Image URL here
        },
        {
            "type": "text", 
            "text": f"""
            Extract the text closest to the item's {entity_name} and return it in 'X unit' format (e.g., '1 cup' or '2.0 grams').
            The 'unit' must either match an entry from the entity_unit_map or be mapped to its equivalent unit based on the context.
            Strictly follow this format: 'X unit' with 'X' as a float or integer.
            """+"""
            entity_unit_map = {
                'width': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
                'depth': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
                'height': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
                'item_weight': {'gram', 'kilogram', 'microgram', 'milligram', 'ounce', 'pound', 'ton'},
                'maximum_weight_recommendation': {'gram', 'kilogram', 'microgram', 'milligram', 'ounce', 'pound', 'ton'},
                'voltage': {'kilovolt', 'millivolt', 'volt'},
                'wattage': {'kilowatt', 'watt'},
                'item_volume': {
                    'centilitre', 'cubic foot', 'cubic inch', 'cup', 'decilitre', 
                    'fluid ounce', 'gallon', 'imperial gallon', 'litre', 'microlitre', 
                    'millilitre', 'pint', 'quart'
                }
            }
            """
        },
    ]
}

        ]
        # Preparation for inference
        text = processor.apply_chat_template(
            messages, tokenize=False, add_generation_prompt=True
        )
        image_inputs, video_inputs = process_vision_info(messages)
        inputs = processor(
            text=[text],
            images=image_inputs,
            videos=video_inputs,
            padding=True,
            return_tensors="pt",
        )
        inputs = inputs.to("cuda")

        # Inference: Generation of the output
        generated_ids = model.generate(**inputs, max_new_tokens=64, temperature=0.01)
        generated_ids_trimmed = [
            out_ids[len(in_ids):] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
        ]
        output_text = processor.batch_decode(
            generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
        )

        # Parse the output
        parsed_output = parse_output(output_text[0], entity_name, entity_unit_map)

        # Store the output in the 'Predictions' column
        test_df.at[index, 'prediction'] = parsed_output

    except Exception as e:
        # If any error occurs, print a message and skip to the next iteration
        print(f"Error processing image at index {index}, URL: {img_url}. Error: {e}")
        continue


  0%|          | 0/10000 [00:00<?, ?it/s]

In [ ]:
# import pandas as pd

# # Example DataFrame (as described by your sample)
# # train_df = pd.DataFrame({...})

# # Check for matches between 'entity_value' and 'Predictions'
# matching_values = test_df['entity_value'][:10000] == test_df['Predictions'][:10000]

# # Count the number of matching rows
# matching_count = matching_values.sum()

# # Display the result
# print(f"Number of matching values between 'entity_value' and 'Predictions': {matching_count}")

# # Optionally, if you want to display the matching rows
# # matching_rows = train_df[matching_values]
# # print("Matching rows:")
# # print(matching_rows)


In [ ]:
# Save the first 10000 rows to a CSV file
test_df.head(10000).to_csv('test10000.csv', index=False)

EDA - Below

In [73]:
train_df['Predictions'].value_counts()[:10]

Predictions
                    263519
100.0 gram              23
200.0 gram              11
1 kilogram               6
150.0 gram               6
1400.0 milligram         6
18.55 gram               5
25.0 gram                4
1.0 volt                 4
100.0 watt               4
Name: count, dtype: int64

In [78]:
import re
import pandas as pd

# Function to check if a number is an integer or float
def is_float_or_integer(value):
    try:
        num = float(value)
        if num.is_integer():
            return 'integer'
        else:
            return 'float'
    except ValueError:
        return None  # Handle non-numeric values

# Function to extract the number and the unit from a string like '500 gram'
def extract_number_and_unit(value):
    match = re.match(r'(\d+\.?\d*)\s*(\w+)', value.strip())
    if match:
        number, unit = match.groups()
        number_type = is_float_or_integer(number)
        return number_type, unit
    return None, None

# Initialize dictionaries to count integers and floats by unit
unit_counts = {'integer': {}, 'float': {}}

# Iterate through each value in the 'Predictions' or 'entity_value' column
for value in train_df['entity_value']:  # Replace 'Predictions' with the column you want
    number_type, unit = extract_number_and_unit(value)
    
    if number_type and unit:
        if unit not in unit_counts[number_type]:
            unit_counts[number_type][unit] = 0
        unit_counts[number_type][unit] += 1

# Convert the result into a DataFrame for easy viewing
integer_count_df = pd.DataFrame.from_dict(unit_counts['integer'], orient='index', columns=['integer_count'])
float_count_df = pd.DataFrame.from_dict(unit_counts['float'], orient='index', columns=['float_count'])

# Merge the two DataFrames to get counts side by side
unit_count_df = pd.merge(integer_count_df, float_count_df, left_index=True, right_index=True, how='outer').fillna(0)

# Display the counts of integers and floats per unit
print(unit_count_df)


             integer_count  float_count
candela               27.0          1.0
carat                  1.0          2.0
centilitre            31.0          1.0
centimetre         47422.0      18245.0
cubic                143.0         70.0
cup                   76.0         11.0
decilitre             94.0          3.0
e                      0.0          6.0
fluid               1471.0        656.0
foot                1006.0        335.0
gallon               167.0          8.0
gigabyte               2.0          0.0
gram               53950.0       9675.0
horsepower           242.0          4.0
inch               17992.0      27489.0
kilogram            8623.0       4597.0
kilowatt              64.0         45.0
litre                675.0        220.0
metre                816.0        714.0
microgram            733.0         15.0
milliampere            1.0          0.0
milligram           7673.0        343.0
millilitre          3440.0        114.0
millimetre         17818.0       1077.0
